In [28]:
dir_base = "../../../"

#### 1. Importação das bibliotecas

In [29]:
from os import listdir, mkdir
from os.path import isdir, isfile, join

In [30]:
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [31]:
from sklearn.metrics import pairwise_distances

#### 2. Definição das funções

In [32]:
def redimensiona_imagens(dir_base):
    
    for pasta in listdir(dir_base+"Dataset/"):
        for arquivo in listdir(dir_base+"Dataset/"+pasta):
            
            imagem = cv2.imread(dir_base+"Dataset/"+pasta+'/'+arquivo)
            imagem_convertida = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
            shape = imagem_convertida.shape
            add_x = int((224-shape[0])/2)
            add_y = int((224-shape[1])/2)
            mapa_distancias = np.zeros((224,224,3),dtype = np.uint8)
            for i in range(3):
                mapa_distancias[:,:,i] = np.pad(imagem_convertida[:,:,i],((add_x,add_x),(add_y,add_y)),'constant', constant_values = (0))
            
            plt.imsave(dir_base+'Dataset/'+pasta+'/'+arquivo, mapa_distancias)
   
    return

In [33]:
def calcula_distancias(coordenadas, dir_base, arquivo):
    
    frame = arquivo.split(".")[0]
    pasta = frame.split("_")[0]
    
    coordenadas = np.array(coordenadas)
    matriz_distancias = pairwise_distances(coordenadas)
       
    if isdir(dir_base+'Dataset/'+pasta):
        plt.imsave(dir_base+'Dataset/'+pasta+'/'+frame+'.png', matriz_distancias)
    else:
        mkdir(dir_base+'Dataset/'+pasta)
        plt.imsave(dir_base+'Dataset/'+pasta+'/'+frame+'.png', matriz_distancias)

    return

In [34]:
def ler_coordenadas(dir_base):
    
    arquivos = [arquivo for arquivo in listdir(dir_base+"Frames/") if isfile(join(dir_base+"Frames/", arquivo))]
    
    for arquivo in arquivos:
        pdb = open(dir_base+"Frames/"+arquivo, "rt")
        linhas = pdb.readlines()
        carbonos_alfa = list(filter(lambda l: l[13:15]=="CA", linhas))
        pdb.close()
        coordenadas = list(map(lambda ca: [float(ca[31:38]), float(ca[39:46]), float(ca[47:54])], carbonos_alfa))  
        calcula_distancias(coordenadas, dir_base, arquivo)
        
    return

In [35]:
ler_coordenadas(dir_base)

In [36]:
redimensiona_imagens(dir_base)